In [50]:
import json 
import time 
from pathlib import Path 

from web3 import Web3
from etherscan import Etherscan
import pandas as pd 

from flywheel_util.constants import url_infura, addresses
from flywheel_util.utils import ddf
from flywheel_util.w3_utils import verified_contract

In [2]:
w3 = Web3(Web3.HTTPProvider(url_infura))

hmmmm = 'VR7YA9DRDB4Y15B5N3WU9E7PSJ9RWPCP5S'
etherscan = Etherscan(hmmmm)

In [3]:
frax = verified_contract(addresses.token.frax)

In [4]:
with Path('frax-collateral-pools.json').open('r') as f: 
    data = json.loads(f.read())
df_pools = pd.DataFrame(data['data'], columns=data['columns'])
df_pools.pool_address = df_pools.pool_address.apply(lambda a: w3.toChecksumAddress(a))
df_pools.head(1)

,contract_address,call_success,call_tx_hash,call_trace_address,call_block_time,call_block_number,pool_address
0,0x853d955acef822db058eb8505911ed77f175b99e,True,0x8a07b19d0919e40a21a60db3c42d82c27176dce7aa61...,[],2021-09-08 00:16:45.000 UTC,13181795,0x2fE065e6FFEf9ac95ab39E5042744d695F560729


In [10]:
pool_addrs = df_pools.pool_address.unique().tolist()
records = []
for a in pool_addrs: 
    contract, address_impl = verified_contract(a, return_address_impl=True)
    collat_usd = contract.functions.collatDollarBalance().call()
    code = etherscan.get_contract_source_code(address_impl) 
    assert len(code) == 1 
    name = code[0]['ContractName']
    r = {'name': name, 'address': a, 'address_impl': address_impl, 'collateral_usd': collat_usd / 1e18}
    records.append(r)
    time.sleep(.1)
df = pd.DataFrame(records)

In [11]:
print(df.collateral_usd.sum())
df.head(10)

1074362368.904034


,name,address,address_impl,collateral_usd
0,FraxPoolV3,0x2fE065e6FFEf9ac95ab39E5042744d695F560729,0x2fE065e6FFEf9ac95ab39E5042744d695F560729,1.009523e+07
1,UniV3LiquidityAMO,0x3814307b86b54b1d8e7B2Ac34662De9125F8f4E6,0x3814307b86b54b1d8e7B2Ac34662De9125F8f4E6,1.133881e+07
2,Pool_USDC,0x1864Ca3d47AaB98Ee78D11fc9DCC5E7bADdA1c0d,0x1864Ca3d47AaB98Ee78D11fc9DCC5E7bADdA1c0d,5.000457e+02
3,FraxAMOMinter,0xcf37B62109b537fa0Cb9A90Af4CA72f6fb85E241,0xcf37B62109b537fa0Cb9A90Af4CA72f6fb85E241,6.454501e+08
4,RariFuseLendingAMO,0x96665d63c1B53f8335e3c9287Ee255f306C93c45,0x96665d63c1B53f8335e3c9287Ee255f306C93c45,1.634531e+07
5,StakeDAO_AMO,0x375278D3C65f29C1A90E8550888f1439cFeFe465,0xcf1e6926b2167f83ec3300bed04a672abd93e646,0.000000e+00
6,Convex_AMO,0x49ee75278820f409ecd67063D8D717B38d66bd71,0x49f77ddd4d57636ab4c98d8f18ca5f4b5210983d,3.911324e+08


In [13]:
frax_amo_minter_address = '0xcf37B62109b537fa0Cb9A90Af4CA72f6fb85E241'
contract = verified_contract(frax_amo_minter_address)
# collateral_addresses = contract.functions.allCollaterals().call()

In [27]:
amo_addrs = [a for a in contract.functions.allAMOAddresses().call() if a != '0x0000000000000000000000000000000000000000'] 
amo_addrs

['0x66635DC1EdEfF19e839f152a67278151Aa6e1B61',
 '0x6e1A844AFff1aa2a8ba3127dB83088e196187110',
 '0x8575FFE80b94fe58c8e0c735E11658A760109f53',
 '0xBA5478A712b5EA898AF03206ab4c7E0608C3e69D',
 '0xddf6b5B2BA110a0267BffB86AeAbFe2637cb8375',
 '0xc7F48Fb6Dbb6F8A3Eed90553017cDf5725Dc44ac',
 '0xF3a21b5d9E11eECA3a50BEb654276987164AbC8d',
 '0x4572B68296A23B4C66696FAd177c50CBc35c532F',
 '0xEc8672dd770D59FaD9a811591a5Edd40e8F6A413',
 '0x977eaDb6fa9b8E1a2A950CcDE1A75a7b527a8cBB',
 '0xc91Bb4b0696e3b48c0C501B4ce8E7244Fc363A79',
 '0x4828D4496Ff8dA2f5c1E7217Dc282b5C69B83263',
 '0x7F35dc487A5422D6946aAD733C6018F163084ed0',
 '0xa5924D9baA4fed0fbd100CB47CBCb61eA5E33219',
 '0x8D0A8f07F5Ea7F0982Ed116993200C331E4cdD27',
 '0xD4f638819Ed2c349E9Cfb6b5168d9e810a9D4f85',
 '0x629C473e0E698FD101496E5fbDA4bcB58DA78dC4',
 '0x0Ed8fA7FC63A8eb5487E7F87CAF1aB3914eA4eCa']

In [34]:
amo_collateral_holdings = []
for amo_addr in amos: 
    # AMO contract 
    c = verified_contract(amo_addr) 
    usd_bal = c.functions.dollarBalances().call()
    correction_offset = contract.functions.correction_offsets_amos(amo_addr, 1).call()
    code = etherscan.get_contract_source_code(amo_addr) 
    assert len(code) == 1 
    name = code[0]['ContractName']
    r = {'name': name, 'usd_balance': (usd_bal[1] + correction_offset) / 1e18, "address": amo_addr}
    print(r) 
    amo_collateral_holdings.append(r)
    time.sleep(.1)

{'name': 'AaveAMO', 'usd_balance': 1880920.535247085, 'address': '0x66635DC1EdEfF19e839f152a67278151Aa6e1B61'}
{'name': 'UniV3LiquidityAMO_V2', 'usd_balance': 4868345.498829985, 'address': '0xc91Bb4b0696e3b48c0C501B4ce8E7244Fc363A79'}
{'name': 'FraxLiquidityBridger_BSC_AnySwap', 'usd_balance': 0.0, 'address': '0x4572B68296A23B4C66696FAd177c50CBc35c532F'}
{'name': 'FraxLiquidityBridger_SOL_WormholeV2', 'usd_balance': 1199.375, 'address': '0xF3a21b5d9E11eECA3a50BEb654276987164AbC8d'}
{'name': 'TWAMM_AMO', 'usd_balance': 2704250.0000000005, 'address': '0x629C473e0E698FD101496E5fbDA4bcB58DA78dC4'}
{'name': 'FraxLiquidityBridger_FTM_AnySwap', 'usd_balance': 31953769.966023847, 'address': '0x8575FFE80b94fe58c8e0c735E11658A760109f53'}
{'name': 'ManualTokenTrackerAMO', 'usd_balance': 500985193.0, 'address': '0xEc8672dd770D59FaD9a811591a5Edd40e8F6A413'}
{'name': 'FraxLiquidityBridger_AUR_Rainbow', 'usd_balance': 2029.125, 'address': '0xD4f638819Ed2c349E9Cfb6b5168d9e810a9D4f85'}
{'name': 'FraxLi

In [51]:
df_amos = pd.DataFrame(amo_collateral_holdings)
df_amos = df_amos.sort_values('usd_balance', ascending=False).reset_index(drop=True) 
last_sync = df.loc[df.name == 'FraxAMOMinter'].collateral_usd.sum()
current = df_amos.usd_balance.sum() # more up to date than last sync so small discrepancies are expected 
print(f"Difference between actual and last sync collateral USDC held by FraxAMOMinter {(current - last_sync) / current:.2%}. Actual: ${current:,}")
ddf(df_amos)

Difference between actual and last sync collateral USDC held by FraxAMOMinter -0.48%. Actual: $642,343,056.8782419


,name,usd_balance,address
0,ManualTokenTrackerAMO,5.009852e+08,0xEc8672dd770D59FaD9a811591a5Edd40e8F6A413
1,FraxlendAMO,4.770818e+07,0x0Ed8fA7FC63A8eb5487E7F87CAF1aB3914eA4eCa
2,FraxLiquidityBridger_FTM_AnySwap,3.195377e+07,0x8575FFE80b94fe58c8e0c735E11658A760109f53
3,FraxLiquidityBridger_POLY_MaticBridge,2.093560e+07,0x6e1A844AFff1aa2a8ba3127dB83088e196187110
4,FraxLiquidityBridger_MOON_AnySwap,1.305618e+07,0xc7F48Fb6Dbb6F8A3Eed90553017cDf5725Dc44ac
5,FraxLiquidityBridger_AVAX_AnySwap,8.231338e+06,0xBA5478A712b5EA898AF03206ab4c7E0608C3e69D
6,FraxLiquidityBridger_BOBA_BobaGateway,6.973608e+06,0x4828D4496Ff8dA2f5c1E7217Dc282b5C69B83263
7,UniV3LiquidityAMO_V2,4.868345e+06,0xc91Bb4b0696e3b48c0C501B4ce8E7244Fc363A79
8,FraxLiquidityBridger_OPTI_Celer,2.797230e+06,0x7F35dc487A5422D6946aAD733C6018F163084ed0
9,TWAMM_AMO,2.704250e+06,0x629C473e0E698FD101496E5fbDA4bcB58DA78dC4
